# Testes MLflow 

### Importações

In [2]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


---

### Base de dados & Data understanding básico

In [3]:
# Leitura da base
dataset_path = '../data/raw/heart.csv'
dataset = pd.read_csv(dataset_path)
# Sanity test - 5 primeiros registros
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [4]:
dataset.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [5]:
dataset.shape

(1025, 14)

In [6]:
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values

In [7]:
X, y

(array([[52.,  1.,  0., ...,  2.,  2.,  3.],
        [53.,  1.,  0., ...,  0.,  0.,  3.],
        [70.,  1.,  0., ...,  0.,  0.,  3.],
        ...,
        [47.,  1.,  0., ...,  1.,  1.,  2.],
        [50.,  0.,  0., ...,  2.,  0.,  2.],
        [54.,  1.,  0., ...,  1.,  1.,  3.]], shape=(1025, 13)),
 array([0, 0, 0, ..., 0, 1, 0], shape=(1025,)))

### Train test split

In [9]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.2, random_state=0) 

In [10]:
# Sanity test - Training Data
X_train.shape, y_train.shape

((820, 13), (820,))

In [11]:
# Sanity test - Test Data
X_test.shape, y_test.shape

((205, 13), (205,))

---

In [35]:
type(y_train)

numpy.ndarray

## Cross validation (GridSearchCV)

In [ ]:
def generic_grid_search_cv(estimator_classifier, param_dict, X_matrix, y_matrix):
    """
    Executa um GridSearchCV para um classificador do Scikit-Learn.

    Esta função recebe um estimador do Scikit-Learn (classificador),
    realiza uma busca em grade com validação cruzada (GridSearchCV) 
    utilizando os hiperparâmetros fornecidos e retorna o nome do 
    estimador e o melhor conjunto de hiperparâmetros encontrado.

    Args:
        estimator_classifier: Estimador Scikit-Learn, como 
            `DecisionTreeClassifier()`, `RandomForestClassifier()`, etc.
        param_dict (dict): Dicionário contendo hiperparâmetros como chaves
            e listas de valores como opções a serem testadas.
        X_matrix (numpy.ndarray): Matriz de features usada para o treinamento.
        y_matrix (numpy.ndarray): Vetor ou matriz de rótulos alvo.

    Returns:
        estimator_name (str): Nome da classe do estimador recebido.
        best_params (dict): Melhor combinação de hiperparâmetros encontrada 
            pelo GridSearchCV.
    """
    
    grid_search = GridSearchCV(estimator=estimator_classifier, param_grid=param_dict)
    grid_search.fit(X = X_matrix, y = y_matrix)
    best_params = grid_search.best_params_
    estimator_name = estimator_classifier.__class__.__name__
    
    return estimator_name, best_params

In [19]:
# Dicionários de parâmetros e valores para efetuar CV

# Decision tree
tree_params = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Random florest
rf_params = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [10, 40, 100, 150],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Gradient boosting classifier
gbc_params = {
    'loss': ['log_loss', 'exponential'],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'n_estimators': [25, 50, 100, 200, 300, 500],
    'criterion': ['friedman_mse', 'squared_error']
}

# K-nearest neighbors classifier
knn_params = {
    'n_neighbors': [3, 5, 10, 20],
    'p': [1, 2]
}

# Logistic regression
lr_params = {
    'tol': [0.0001, 0.00001, 0.000001],
    'C': [1.0, 1.5, 2.0],
    'solver': ['lbfgs', 'sag', 'saga']
}

# Support vector machine
svm_params = {
    'tol': [0.001, 0.0001, 0.00001],
    'C': [1.0, 1.5, 2.0],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

# Neural network classifier
neural_net_params = {
    'activation': ['relu', 'logistic', 'tahn'],
    'solver': ['adam', 'sgd'],
    'batch_size': [10, 56]
}

In [41]:
model_name, tree_best_params = generic_grid_search_cv(DecisionTreeClassifier(), param_dict=tree_params, X_matrix=X, y_matrix=y)
print(f'Model: {model_name}. Best params{tree_best_params}')

Model: DecisionTreeClassifier. Best params{'criterion': 'gini', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


In [40]:
model_name, rf_best_params = generic_grid_search_cv(RandomForestClassifier(),param_dict=rf_params, X_matrix=X, y_matrix=y)
print(f'Model: {model_name}. Best params{rf_best_params}')

Model: RandomForestClassifier. Best params{'criterion': 'gini', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 40}


In [ ]:
model_name, gbc_best_params = generic_grid_search_cv(GradientBoostingClassifier(), param_dict=gbc_params, X_matrix=X, y_matrix=y)
print(f'Model: {model_name}. Best params{gbc_best_params}')

Model: GradientBoostingClassifier. Best params{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 300}


---

Dicionários de melhor combinação de parâmetros dos modelos, definida pelo GridSearchCV (Cross validation).

In [ ]:
tree_params = {
    'criterion': 'entropy', 
    'min_samples_leaf':1, 
    'min_samples_split': 5,
    'splitter':'best'
}

rf_params = {
    'criterion': 'entropy',
    'min_samples_leaf': 1,
    'n_estimators': 40
}

gb_params = {
    'criterion': 'friedman_mse', 
    'learning_rate': 0.1,
    'loss': 'exponential',
    'n_estimators': 300
}

knn_params = {
    'n_neighbors': 20,
    'p': 1
}

lr_params = {
    'C': 1.0,
    'solver': 'lbfgs',
    'tol': 0.0001
}

svm_params = {
    'C': 1.5,
    'kernel': 'rbf',
    'tol': 0.001
}

nn_params = {
    'activation': 'relu',
    'batch_size': 10,
    'solver': 'adam'
}


Treinamento dos modelos com os parâmetros especificados

In [ ]:
tree_classifier = DecisionTreeClassifier(**tree_params)
tree_classifier.fit(X_train, y_train)

rf_classifier = RandomForestClassifier(**rf_params)
rf_classifier.fit(X_train, y_train)

gb_classifier = GradientBoostingClassifier(**gb_params)
gb_classifier.fit(X_train, y_train)

knn_classifier = KNeighborsClassifier(**knn_params)
knn_classifier.fit(X_train, y_train)

lr_classifier = LogisticRegression(**lr_params)
lr_classifier.fit(X_train, y_train)

svm_classifier = SVC(**svm_params)
svm_classifier.fit(X_train, y_train)

nn_classifier = MLPClassifier(**nn_params)
nn_classifier.fit(X_train, y_train)

/home/gustavoalves/Documents/Github/mlops_project/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,hidden_layer_sizes,"(100,)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,10
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,None


Efetuando previsões

In [9]:
y_pred_tree = tree_classifier.predict(X_test) 
y_pred_rf = rf_classifier.predict(X_test)
y_pred_gb = gb_classifier.predict(X_test)
y_pred_knn = knn_classifier.predict(X_test)
y_pred_lr = lr_classifier.predict(X_test)
y_pred_svm = svm_classifier.predict(X_test)
y_pred_nn = nn_classifier.predict(X_test)

---

### Decision Tree

Obtendo métricas do modelo

In [10]:
accuracy = accuracy_score(y_test, y_pred_tree)
precision = precision_score(y_test, y_pred_tree)
recall = recall_score(y_test, y_pred_tree)
f1 = f1_score(y_test, y_pred_tree)


In [11]:
# from sklearn.metrics import accuracy_score as acc_score, precision_score as prec_score, recall_score as rec_score, f1_score

def metrics (y_test, y_pred_model):
    accuracy_metric = accuracy_score(y_test, y_pred_model)
    precision_metric = precision_score(y_test, y_pred_model)
    recall_metric = recall_score(y_test, y_pred_model)
    f1_metric = f1_score(y_test, y_pred_model)

    return accuracy_metric, precision_metric, recall_metric, f1_metric

In [12]:
print(metrics(y_test, y_pred_tree))

(1.0, 1.0, 1.0, 1.0)


## MLflow

### Tracking URI
Esse é o endereço central de onde os experimentos (e consequentemente os modelos) ficarão registrados e serão acessados via MLflow UI.

In [24]:
# Definir o tracking ui
mlflow.set_tracking_uri(uri='http://localhost:5000')

### Título do experimento
O experimento é como o repositório que englobará todos os modelos.

In [25]:
# Definindo título do experimento 
mlflow.set_experiment('Heart disease experiment')

<Experiment: artifact_location='mlflow-artifacts:/577837039507156569', creation_time=1751576336739, experiment_id='577837039507156569', last_update_time=1751576336739, lifecycle_stage='active', name='Heart disease experiment', tags={}>

### Registro de modelos

#### Decision Tree

Registrando modelo (Decision Tree Classifier) com seus parâmetros, métricas e assinatura

In [14]:
# Registro do modelo de arvore de decisão e seus dados

with mlflow.start_run():
    # Método para registro de parâmetros do modelo
    mlflow.log_params(tree_params)
    
    # Métodos para registro de métricas do modelo
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    # Definição de tag para o modelo na UI
    mlflow.set_tag('Training info', 'Basic Decision tree for heart disease data')

    # Registro da signature
    signature = infer_signature(X_train, tree_classifier.predict(X_test))
    
    # Registro do modelo
    model_info = mlflow.sklearn.log_model(
        # 
        sk_model=tree_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name="Decision-tree-classifier"
    )
    
    print(f'Caminho do artefato registrado: {model_info.artifact_path}')
    print(f'URI do modelo registrado: {model_info.model_uri}')

Registered model 'Decision-tree-classifier' already exists. Creating a new version of this model...
2025/11/18 16:39:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Decision-tree-classifier, version 5


Caminho do artefato registrado: mlflow-artifacts:/577837039507156569/models/m-4509a65d23194aa8b5b139720d54c1e5/artifacts
URI do modelo registrado: models:/m-4509a65d23194aa8b5b139720d54c1e5
🏃 View run skittish-kite-734 at: http://localhost:5000/#/experiments/577837039507156569/runs/f7dd82319fcd4e0ea5ed20740fcf9d06
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '5' of model 'Decision-tree-classifier'.


---


## Testes na etapa de registro
### Random forest 

In [30]:
X_train

array([[42.,  1.,  3., ...,  2.,  2.,  2.],
       [66.,  0.,  2., ...,  1.,  1.,  2.],
       [53.,  1.,  2., ...,  2.,  3.,  2.],
       ...,
       [65.,  1.,  3., ...,  1.,  1.,  2.],
       [67.,  1.,  0., ...,  1.,  0.,  2.],
       [60.,  1.,  2., ...,  1.,  0.,  2.]], shape=(820, 13))

In [40]:

mlflow_dataset = mlflow.data.from_pandas(
    dataset, source=dataset_path, name='heart-disease-dataset'
)

with mlflow.start_run():
    mlflow.log_params(rf_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_rf)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    
    mlflow.set_tag('Training info', 'Basic Random forest for heart disease data')
    mlflow.set_tag('Scientist info', 'Test Test Test aaaaaaaaaaaaaa')
    
    signature = infer_signature(X_train, y_pred_rf)
    
    model_info = mlflow.sklearn.log_model(
        sk_model=rf_classifier,
        name='rf_heart_model',
        input_example=X_train[[0]],
        registered_model_name="Random-forest-classifier",
    )
    
    mlflow.log_input(mlflow_dataset, context='training')
    
print('-------------------------------------------')
print("Model URI: ", model_info.model_uri)
print("Model ID: ", model_info.model_id)
print("Artifact Path: ", model_info.artifact_path)
print("Model version: ", model_info.registered_model_version)
print('-------------------------------------------')

/home/gustavoalves/Documents/Github/mlops_project/venv/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/raw/heart.csv'. Exception: 
  return _dataset_source_registry.resolve(
/home/gustavoalves/Documents/Github/mlops_project/venv/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
Registered model 'Random-forest-classifier' already exists. Creating a new version of this model...
2025/11/18 21:09:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random-forest-classifier, version 14


🏃 View run colorful-shark-49 at: http://localhost:5000/#/experiments/577837039507156569/runs/73e2eebcc23046ee8f47429c63b1a8f6
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569
-------------------------------------------
Model URI:  models:/m-dec311af760041b3af732ecf481e3ff0
Model ID:  m-dec311af760041b3af732ecf481e3ff0
Artifact Path:  mlflow-artifacts:/577837039507156569/models/m-dec311af760041b3af732ecf481e3ff0/artifacts
Model version:  14
-------------------------------------------


Created version '14' of model 'Random-forest-classifier'.
/home/gustavoalves/Documents/Github/mlops_project/venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


---

### Gradient boosting

In [11]:
with mlflow.start_run():
    mlflow.log_params(gb_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_gb)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    mlflow.set_tag('Training info', 'Basic Gradient boosting for heart disease data')
    signature = infer_signature(X_train, gb_classifier.predict(X_test))
    
    model_info = mlflow.sklearn.log_model(
        sk_model=gb_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name="Gradient-boosting-classifier"
    )
    print("Model URI", model_info.model_uri)

Successfully registered model 'Gradient-boosting-classifier'.
2025/07/15 14:55:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Gradient-boosting-classifier, version 1


Model URI models:/m-aee88f2d36d14ac3bd3f8882426b6aa7
🏃 View run treasured-sponge-312 at: http://localhost:5000/#/experiments/577837039507156569/runs/c25d58de82bc4782bd293a8776f87263
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'Gradient-boosting-classifier'.


#### KNN

In [12]:
import mlflow.sklearn

with mlflow.start_run():
    mlflow.log_params(knn_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_knn)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    mlflow.set_tag('Training info', 'Basic KNN model for heart disease data')
    signature = infer_signature(X_train, knn_classifier.predict(X_test)) 
    
    model_info = mlflow.sklearn.log_model(
        sk_model=knn_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name='KNN-classifier'
    )
    print("Model URI", model_info.model_uri)

Successfully registered model 'KNN-classifier'.
2025/07/15 14:55:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: KNN-classifier, version 1


Model URI models:/m-a3656752e8744086b1babaa1d56115d2
🏃 View run languid-finch-308 at: http://localhost:5000/#/experiments/577837039507156569/runs/70422ff2bb6d4b72a6f2acb04935dac5
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'KNN-classifier'.


#### Logistic regression

In [13]:
with mlflow.start_run():
    mlflow.log_params(lr_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_lr)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    mlflow.set_tag('Training info', 'Basic Logistic regression for heart disease data')
    signature = infer_signature(X_train, lr_classifier.predict(X_test))
    
    model_info = mlflow.sklearn.log_model(
        sk_model=lr_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name='Logistic-regression-classifier'
    )
    
    print('Model URI', model_info.model_uri)

Successfully registered model 'Logistic-regression-classifier'.
2025/07/15 14:55:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic-regression-classifier, version 1


Model URI models:/m-e1406d4f268e4cf68539008fcbccf4cc
🏃 View run popular-hare-443 at: http://localhost:5000/#/experiments/577837039507156569/runs/5858fbdf396b4d72a70b0825b48ba2a8
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'Logistic-regression-classifier'.


#### SVM

In [14]:
with mlflow.start_run():
    mlflow.log_params(svm_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_svm)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    mlflow.set_tag('Training info', 'Basic SVM model for heart disease data')
    signature = infer_signature(X_train, svm_classifier.predict(X_test))
    
    model_info = mlflow.sklearn.log_model(
        sk_model=svm_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name='SVM-classifier'
    )
    
    print('Model URI', model_info.model_uri)

Successfully registered model 'SVM-classifier'.
2025/07/15 14:55:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SVM-classifier, version 1


Model URI models:/m-6459c453dd6c4299a5cb30b82ba99a99
🏃 View run thoughtful-gnu-119 at: http://localhost:5000/#/experiments/577837039507156569/runs/98bf40577e6c44dbbf564d1e34e28335
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'SVM-classifier'.


#### Neural network

In [15]:
with mlflow.start_run():
    mlflow.log_params(nn_params)
    
    accuracy, precision, recall, f1 = metrics(y_test, y_pred_nn)
    
    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('f1 score', f1)
    
    mlflow.set_tag('Training info', 'Basic Neural network model for heart disease data')
    signature = infer_signature(X_train, nn_classifier.predict(X_test))
    
    model_info = mlflow.sklearn.log_model(
        sk_model=nn_classifier,
        signature=signature,
        input_example=X_test,
        registered_model_name='Neural-network-classifier'
    )
    
    print('Model URI', model_info.model_uri)

Successfully registered model 'Neural-network-classifier'.
2025/07/15 14:55:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Neural-network-classifier, version 1


Model URI models:/m-f99e754a1de748ba8a4294e776690741
🏃 View run unruly-lark-447 at: http://localhost:5000/#/experiments/577837039507156569/runs/126f2cd2fd27464091ef280a39ae246a
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'Neural-network-classifier'.


---

Função genérica para registro de modelos

In [4]:
def train_and_log_model (classifier, X_train, X_test, y_train, y_test, **params):
    classifier_name = classifier.__name__
    classifier = classifier(**params)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    with mlflow.start_run():
        mlflow.log_params(params)
        
        accuracy, precision, recall, f1 = metrics(y_test, y_pred)
        
        mlflow.log_metric('Accuracy', accuracy)
        mlflow.log_metric('Precision', precision)
        mlflow.log_metric('Recall', recall)
        mlflow.log_metric('f1 score', f1)
        
        mlflow.set_tag('Training info', f'Basic {classifier_name}  model for heart disease data')
        signature = infer_signature(X_train, classifier.predict(X_test))
        
        model_info = mlflow.sklearn.log_model(
            sk_model=classifier,
            signature=signature,
            input_example=X_test,
            registered_model_name=classifier_name
        )
        
        print('Model URI', model_info.model_uri)

In [17]:
mlflow.set_tracking_uri(uri='http://localhost:5000')
mlflow.set_experiment('Heart disease experiment')
train_and_log_model(DecisionTreeClassifier, X_train, X_test, y_train, y_test, **tree_params)

Successfully registered model 'DecisionTreeClassifier'.
2025/07/17 13:37:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DecisionTreeClassifier, version 1


Model URI models:/m-44b5583cc9d941be84b72ffd60b0818a
🏃 View run sincere-quail-760 at: http://localhost:5000/#/experiments/577837039507156569/runs/34a020e08ab64947be17aaedb5e9f06c
🧪 View experiment at: http://localhost:5000/#/experiments/577837039507156569


Created version '1' of model 'DecisionTreeClassifier'.
